# Matematikai programcsomagok - Negyedik gyakorlat (SAT)

## SAT - Logikai kielégíthetőség

| Matematikai jelölés                     | Megnevezés             | PySAT szintaxis          |
| :-- |    :----:              |                ---:      |
| $x_1$                                   | Literál                |       `1`                |
| $\lnot x_1$                             | Literál negáltja       |       `-1`               |
| $\left(x_1\lor x_2\lor\lnot x_3\right)$ | Klóz (clause)          |       `[1,2,-3]`         |
| $(x_1\lor x_2)\land(x_1\lor\lnot x_3)$  | Konjunktív normálforma | `CNF([[1, 2], [1, -3]])` |

Engedélyezett műveletek egy konjunktív normálformában (**CNF**):
* Literálok és tagadásuk "össze-**VAGY**-olása" klózokká,
* Klózok "össze-**ÉS**-elése".

Egy CNF ezen két művelet felhasználásával létrejövő logikai formula. A SAT feladat annak eldöntése, hogy tudunk-e olyan értéket adni a literáloknak (**IGAZ** vagy **HAMIS**), amelyek mellett a logikai formula **IGAZ**-ra értékelődik ki.

---

__Állítás:__ Tetszőleges logikai formula átalakítható konjunktív normálformára.

---

__Igazságtábla:__ Egy olyan táblázat, ahol felsoroljuk az összes lehetséges értékét a literáloknak ($n$ literál esetén ez $2^n$ lehetséges kiértékelés) illetve az adott értékek mellett a logikai formula értékét. Ennek segítségével tetszőleges logikai formula CNF-fé alakítható.

Például vegyük azt a kifejezést, hogy

$$\phi\colon = x_1 \land \lnot x_2\implies x_3 \land\lnot x_1.$$

Ez nyilvánvalóan nem CNF. Írjuk fel az igazságtábláját:

| $x_1$  | $x_2$  | $x_3$  | $\phi$ |
| :----: | :----: | :----: | :----: |
| $0$    | $0$    | $0$    | $1$    |
| $0$    | $0$    | $1$    | $1$    |
| $0$    | $1$    | $0$    | $1$    |
| $0$    | $1$    | $1$    | $1$    |
| $1$    | $0$    | $0$    | $0$    |
| $1$    | $0$    | $1$    | $0$    |
| $1$    | $1$    | $0$    | $1$    |
| $1$    | $1$    | $1$    | $1$    |

_(itt az egyszerűség kedvéért az **IGAZ** értéket $1$-gyel, a **HAMIS**-at $0$-val jelöltem.)_

Ez CNF-ként az alábbi lenne:

$$
    (\lnot x_1\lor x_2\lor x_3)\land(\lnot x_1\lor x_2\lor\lnot x_3).
$$

Az eljárás lényege:
* Kiválasztjuk azokat a sorokat, amelyekben **HAMIS** a formula értéke
* Ennek a sornak minden literálját tagadjuk, és csinálunk belőle egy klózt
* Az így kapott klózokat össze-**ÉS**-eljük.

## PySAT

In [20]:
from pysat.formula import CNF
from pysat.solvers import Glucose3
from pysat.formula import IDPool

In [37]:
# Create an empty CNF and add clauses
formula = CNF()
formula.append([1,2,3])
formula.append([-1,2,-3])

formula.clauses

[[1, 2, 3], [-1, 2, -3]]

In [47]:
formula = CNF()
formula.extend([[1,2,3], [-1,2,-3]])

formula.clauses

[[1, 2, 3], [-1, 2, -3]]

In [48]:
dir(formula)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'append',
 'clauses',
 'comments',
 'copy',
 'extend',
 'from_aiger',
 'from_clauses',
 'from_file',
 'from_fp',
 'from_string',
 'negate',
 'nv',
 'to_alien',
 'to_dimacs',
 'to_file',
 'to_fp',
 'weighted']

In [38]:
# Initialize solver instance, add clauses and solve it
g = Glucose3()
g.add_clause([1,2,3])
g.add_clause([-1,2,-3])
g.solve(), g.get_model()

(True, [1, -2, -3])

In [24]:
# Initialize solver and add clauses from a formula
g = Glucose3()
for c in formula.clauses:
    g.add_clause(c)

g.solve(), g.get_model()

(True, [1, -2, -3])

In [25]:
# Initialize solver with CNF and solve it
g = Glucose3(formula)

g.solve(), g.get_model()

(True, [1, -2, -3])

In [46]:
# Initialize solver with CNF and solve it with fixed variables
g = Glucose3(formula)

g.solve(assumptions=[-1,2]), g.get_model()

(True, [-1, 2, -3])

In [27]:
# Variable IDs
vpool = IDPool()
x = lambda i: vpool.id(f'x_{i}')

formula = CNF()
for i in range(5):
    for j in range(i+1, 6):
        formula.append([x(i), x(j)])
        
print([[vpool.obj(i) for i in clause] for clause in formula.clauses])

[['x_0', 'x_1'], ['x_0', 'x_2'], ['x_0', 'x_3'], ['x_0', 'x_4'], ['x_0', 'x_5'], ['x_1', 'x_2'], ['x_1', 'x_3'], ['x_1', 'x_4'], ['x_1', 'x_5'], ['x_2', 'x_3'], ['x_2', 'x_4'], ['x_2', 'x_5'], ['x_3', 'x_4'], ['x_3', 'x_5'], ['x_4', 'x_5']]


## Telepítés

https://pysathq.github.io/index.html

__Telepítés után újra kell indítani a kernelt!__

In [21]:
!pip install --user python-sat

## Feladatok

### 1) Bemelegítés
Legyen $x_1,x_2$ és $x_3$ logikai változók, írj fel egy olyan $\phi(x_1,x_2,x_3)$ logikai formulát, amely pontosan akkor igaz, ha
* legfeljebb kettő változó igaz,
* legfeljebb egy változó igaz,
* minden változó igaz,
* pontosan az egyik változó igaz.

In [28]:
at_most_two = CNF()


In [29]:
at_most_one = CNF()


In [30]:
same = CNF()


In [31]:
exactly_one = CNF()


### 2) "Pontosan egy", "legfeljebb egy" segédfüggvények
Írj egy függvényt, amely klózoknak egy olyan listáját adja vissza, amely CNF-ként értelmezve pontosan akkor igaz, ha pontosan egy literál igaz/ legfeljebb egy literál igaz.

_(Hint: az egyiket felhasználva könnyen megkapható a másik.)_

In [32]:
def at_most_one(lits):
    '''
        Given a list of literals the function creates the clauses of the CNF
        that is true iff at most one literal is true
    '''
    # TODO
    return clauses

In [33]:
def exactly_one(lits):
    '''
        Given a list of literals the function creates the clauses of the CNF
        that is true iff only one literal is true
    '''
    # TODO
    return clauses

### 3) Igazságtábla

Írj egy függvényt, amelynek bemenete egy CNF, és kiírja a kapott CNF igazságtábláját!

In [34]:
import itertools

def truth_table(cnf):
    '''
        Compute all possible variations of truth values of literals,
        evaluate the formula and print the truth table
    '''
    # TODO

### 4) $n$-királynő probléma

Oldd meg az $n$-királynő problémát SAT feladatként! Az $n$-királynő problémában egy $n\times n$ méretű sakktáblára szeretnénk elhelyezni $n$ királynőt úgy, hogy semelyik kettő ne tudja ütni egymást.
* Modellezd SAT problémaként,
* oldd meg,
* ábrázold az így kapott sakktáblát

_(Hint: a korábban megírt függvények még hasznosak lehetnek)_

In [35]:
from pysat.card import CardEnc, EncType
def nqueens(n):
    '''
        Solve the n queens problem modeled as SAT
        input:
            n (int) : number of queens            
    '''
    # TODO

In [36]:
def print_nqueens(sol):
    '''
        Print solution of the n-queens problem
    '''
    # TODO

### 5) Sudoku
Oldd meg a Sudoku feladatot SAT problémaként! Hogyan kellene megoldani, ha adott előre néhány mező tartalma? 
* Modellezd SAT-ként,
* oldd meg,
* rajzold ki a kapott táblát!